In [25]:
import os
import pandas as pd
import openai
import tiktoken
from PyPDF2 import PdfReader
from tqdm import tqdm
from dotenv import load_dotenv

In [26]:
# 讀取 .env 設定
load_dotenv()
openai.api_key = os.environ['OPENAI_API_KEY']

In [27]:
# 讀取 Excel 檔案，包含 91 個問題及其標籤
excel_path = "data/tcfd第四層接露指引.xlsx"
df_questions = pd.read_excel(excel_path)

In [28]:
df_questions

,Label,Definition
0,G-1-1_1,公司是否描述向董事會和/或董事會下設委員會，定期報告氣候相關風險與機會之流程？
1,G-1-1_2,公司是否描述向董事會和/或董事會下設委員會，報告氣候相關風險與機會之頻率？
2,G-1-2_3,公司是否描述董事會和/或董事會下設委員會，在監督和指導策略、重要行動計畫、風險管理政策、年度...
3,G-1-2_4,公司是否描述董事會和/或董事會下設委員會，在監督重要資本支出、企業收購和撤資時，考量氣候相關...
4,G-1-3_5,公司是否描述董事會或其所屬和/或指派之委員會如何監督風險與機會目標之實現？
...,...,...
86,MT-3-2_78,公司設定目標時，是否有描述衡量進度情況的基準年份？
87,MT-3-2_79,公司設定目標時，是否有描述評估目標進展情況的重要績效指標？
88,MT-3-3_80,公司是否依總體或業務劃分揭露中期目標？
89,MT-3-3_81,公司是否依總體或業務劃分揭露長期目標？


In [29]:
# 假設 Excel 檔案有 'Label' 和 'Question' 欄位
questions = df_questions[['Label', 'Definition']].dropna().values.tolist()

In [30]:
questions

[['G-1-1_1', '公司是否描述向董事會和/或董事會下設委員會，定期報告氣候相關風險與機會之流程？'],
 ['G-1-1_2', '公司是否描述向董事會和/或董事會下設委員會，報告氣候相關風險與機會之頻率？'],
 ['G-1-2_3',
  '公司是否描述董事會和/或董事會下設委員會，在監督和指導策略、重要行動計畫、風險管理政策、年度預算和營業計畫、公司的營業目標、計畫實施和執行情況時，考量氣候相關風險與機會？'],
 ['G-1-2_4', '公司是否描述董事會和/或董事會下設委員會，在監督重要資本支出、企業收購和撤資時，考量氣候相關風險與機會？'],
 ['G-1-3_5', '公司是否描述董事會或其所屬和/或指派之委員會如何監督風險與機會目標之實現？'],
 ['G-2-1_6', '公司是否描述已分派氣候相關責任給管理職位或委員會？'],
 ['G-2-1_7', '公司是否描述該管理職位或委員會向董事會或董事會下設委員會進行報告？'],
 ['G-2-1_8', '公司是否描述已分派之職責包含評估和/或管理氣候相關風險與機會？'],
 ['G-2-2_9', '公司是否有對氣候相關組織結構之描述？'],
 ['G-2-2_10', '公司是否描述有跨部門之氣候相關工作小組統籌執行相關工作？'],
 ['G-2-3_11', '公司是否描述執行管理階層掌握氣候相關風險與機會之流程？'],
 ['G-2-4_12', '公司是否提供管理階層透過特定職位監控氣候相關風險與機會之說明？'],
 ['G-2-4_13', '公司是否提供管理階層透過委員會監控氣候相關風險與機會之說明？'],
 ['S-1-1_14', '公司是否描述「短期」的氣候相關風險與機會，並考量其資產或基礎設施的使用壽命？'],
 ['S-1-1_15',
  '公司是否描述「中期」的氣候相關風險與機會，並考量其資產或基礎設施的使用壽命，以及中期對公司與供應鏈潛在的氣候相關風險與機會？'],
 ['S-1-1_16',
  '公司是否描述「長期」的氣候相關風險與機會，並考量其資產或基礎設施的使用壽命，以及長期對公司與供應鏈潛在的氣候相關風險與機會？'],
 ['S-1-2_17', '公司應描述在「短期」的時間長度下，可能會產生重大財務影響的氣候相關風險與機會？'],
 ['S-1

In [31]:
# 設定 GPT 模型的 Token 限制
MODEL_NAME = "gpt-4o-mini"
MAX_TOKENS = 4096  # OpenAI 最大 Token 限制（請根據 API 規格調整）
OVERLAP_TOKENS = 50  # Overlap token 數量

In [32]:
# 初始化 Token 計數器
tokenizer = tiktoken.encoding_for_model(MODEL_NAME)

In [36]:
def count_tokens(text):
    return len(tokenizer.encode(text))

def split_text(text, max_tokens, overlap_tokens):
    """根據 Token 限制分割文本，確保有重疊部分"""
    words = text.split()
    chunks = []
    start = 0

    while start < len(words):
        end = start + max_tokens
        chunk = " ".join(words[start:end])
        chunks.append(chunk)
        start = end - overlap_tokens  # Overlap 設定

    return chunks

def extract_text_from_pdf(pdf_path):
    """從 PDF 提取文字"""
    with open(pdf_path, "rb") as file:
        reader = PdfReader(file)
        text = "\n".join([page.extract_text() for page in reader.pages if page.extract_text()])
    return text

def query_gpt(chunk, question):
    prompt = f"""
    Here is a text chunk from a TCFD report:

    {chunk}

    Question: {question}
    Does this text provide a relevant answer to the question? Answer with "Yes" or "No" only.
    """
    try:
        response = openai.chat.completions.create(
            model=MODEL_NAME,
            messages=[
                {"role": "system", "content": "You are an expert in climate-related financial disclosures."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=10
        )
        # Access the message content
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error querying OpenAI: {e}")
        return "Error"

In [37]:
# 設定 PDF 來源資料夾
pdf_folder = "data/tcfd_report_pdf_for_testing"
output_data = []

In [39]:
# 遍歷所有 PDF 報告
for pdf_file in tqdm(os.listdir(pdf_folder)):
    if not pdf_file.endswith(".pdf"):
        continue

    pdf_path = os.path.join(pdf_folder, pdf_file)
    text = extract_text_from_pdf(pdf_path)

    # 分割文本
    chunks = split_text(text, MAX_TOKENS, OVERLAP_TOKENS)

    for chunk_id, chunk in enumerate(chunks):
        for label, question in questions:
            answer = query_gpt(chunk, question)
            output_data.append([pdf_file, chunk_id, label, question, answer])

100%|██████████| 6/6 [1:02:42<00:00, 627.14s/it]


In [42]:
# 儲存結果到 CSV
output_df = pd.DataFrame(output_data, columns=["Report", "Chunk ID", "Label", "Question", "Answer"])
output_df.to_csv("data/llm_question_answering_results/tcfd_gpt_results.csv", index=False)

print("處理完成，結果已儲存至 data/llm_question_answering_results/tcfd_gpt_results.csv")

處理完成，結果已儲存至 data/llm_question_answering_results/tcfd_gpt_results.csv
